In [1]:
%load_ext autoreload
%autoreload 2

import sys

# optimus
sys.path.append("..")

# pypika
sys.path.append("../../pypika")

In [2]:
from pypika import Query, Table, Field, analytics as an
from pypika.dialects import MySQLQuery as Query

In [3]:
agg = [an.Min('RATING'),an.Max('RATING')]
q = Query.from_('main.music').select(*agg)
print(q.get_sql())

SELECT MIN('RATING'),MAX('RATING') FROM `main.music`


In [4]:
"SELECT MIN(RATING) FROM main.music"

'SELECT MIN(RATING) FROM main.music'

In [4]:
from pypika import functions as fn
from pypika import Tables, MySQLQuery, Interval

fruits, consumers = Tables('fruits', 'consumers')
q = MySQLQuery.from_(fruits) \
    .join(consumers) \
    .on(fruits.consumer_id == consumers.id) \
    .select(fruits.id, fruits.name) \
    .where((fruits.harvest_date + Interval(days=1)) < fn.Now())
print(q.get_sql())

SELECT `fruits`.`id`,`fruits`.`name` FROM `fruits` JOIN `consumers` ON `fruits`.`consumer_id`=`consumers`.`id` WHERE `fruits`.`harvest_date`+INTERVAL 1 DAY<NOW()


In [3]:
from optimus import Optimus
op= Optimus(master="local", app_name= "optimus", verbose = True)

C:\Users\argenisleon\Anaconda3\lib\site-packages\socks.py:58: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Callable

    You are using PySparkling of version 2.4.10, but your PySpark is of
    version 2.3.1. Please make sure Spark and PySparkling versions are compatible. 
INFO:optimus:Operative System:Windows
INFO:optimus:Just check that Spark and all necessary environments vars are present...
INFO:optimus:-----
INFO:optimus:SPARK_HOME=C:\opt\spark\spark-2.3.1-bin-hadoop2.7
INFO:optimus:HADOOP_HOME=C:\opt\hadoop-2.7.7
INFO:optimus:PYSPARK_PYTHON=C:\Users\argenisleon\Anaconda3\python.exe
INFO:optimus:PYSPARK_DRIVER_PYTHON=jupyter
INFO:optimus:PYSPARK_SUBMIT_ARGS=--jars "file:///C:/Users/argenisleon/Documents/Optimus/optimus/jars/RedshiftJDBC42-1.2.16.1027.jar,file:///C:/Users/argenisleon/Documents/Optimus/optimus/jars/mysql-connector-java-8.0.16.jar,file:///C:

In [6]:
df = op.load.csv("data/Meteorite_Landings.csv").h_repartition()

In [11]:
df.show()

+--------------------+-----+--------+--------+--------+-----+--------------------+----------+----------+--------------------+
|                name|   id|nametype|recclass|mass (g)| fall|                year|    reclat|   reclong|         GeoLocation|
+--------------------+-----+--------+--------+--------+-----+--------------------+----------+----------+--------------------+
|Dominion Range 08357|52132|   Valid|      L6|     8.9|Found|01/01/2008 12:00:...|       0.0|       0.0|(0.000000, 0.000000)|
|       Yamato 792863|28212|   Valid|      H5|  132.25|Found|01/01/1979 12:00:...|     -71.5|  35.66667|(-71.500000, 35.6...|
|           Acfer 232|  240|   Valid|      H5|   725.0|Found|01/01/1991 12:00:...|  27.73944|   4.32833|(27.739440, 4.328...|
|Jiddat al Harasis...|56470|   Valid|      L5|    17.5|Found|                null|  19.83528|  56.46139|(19.835280, 56.46...|
|               Imlay|52855|   Valid|      L5|   770.0|Found|01/01/2009 12:00:...|  40.74018|-118.17285|(40.740180, -1

In [40]:
from pypika import Tables, MySQLQuery, Interval

df.set_name("df")

agg = [an.Min('id').as_('a'),an.Max('id')]

q = MySQLQuery.from_("df").select(*agg) 


def clean(c):
    return q.get_sql().replace("'","`")

df.query(clean(q)).show()

+---+-------+
|  a|max(id)|
+---+-------+
|  1|  57458|
+---+-------+



In [45]:
df.cols.min_sql("id")

MIN('id')
SELECT MIN(`id`) `Min_id` FROM `df`
+------+
|Min_id|
+------+
|     1|
+------+



AssertionError: all exprs should be Column

In [93]:
df.cols.max("id")

57458

In [95]:
df.cols.dtypes()

{'name': 'string',
 'id': 'int',
 'nametype': 'string',
 'recclass': 'string',
 'mass (g)': 'double',
 'fall': 'string',
 'year': 'string',
 'reclat': 'double',
 'reclong': 'double',
 'GeoLocation': 'string'}

In [96]:
df.cols.count_zeros(["id","mass (g)"])

{'id': 0, 'mass (g)': 19}

In [111]:
df.cols.count_uniques(["id","mass (g)"])

{'id': 42365, 'mass (g)': 12497}

In [116]:
df.query("SELECT COUNT(*), recclass from df GROUP BY recclass ORDER BY COUNT(*) DESC").show()

+--------+-----------+
|count(1)|   recclass|
+--------+-----------+
|    8285|         L6|
|    7142|         H5|
|    4796|         L5|
|    4528|         H6|
|    4211|         H4|
|    2766|        LL5|
|    2043|        LL6|
|    1253|         L4|
|     428|       H4/5|
|     416|        CM2|
|     386|         H3|
|     365|         L3|
|     335|        CO3|
|     300|   Ureilite|
|     285|Iron, IIIAB|
|     268|        LL4|
|     256|        CV3|
|     241|  Diogenite|
|     240|  Howardite|
|     225|         LL|
+--------+-----------+
only showing top 20 rows

